# The impact of location in restaurant ratings on social media

## 1. Introduction

##### Description of the problem and a discussion of the background

When a user rates a restaurant on Foursquare he surely takes into account the quality of the food, of the service, the setting but __does the location of a restaurant impact its rating ?__

The fact that a restaurant is well located, that is to say easy to access, should increase the number of comments but not necessarily impact the ratings significantly.

In this study, we will see if a correlation exists between the location of a restaurant and its global score on Foursquare. 

__Target audience :__
If this correlation exists, this study will show the most popular areas in NYC to open a restaurant. This map will help entrepreneurs who want to open a restaurant in NYC, providing guidance on which area to choose to settle their restaurant.

## 2. Data

##### Description of the data and how it will be used to solve the problem

To answer this question, we will analyze the Foursquare ratings of NYC restaurants. For that, we will use the Foursquare API to get the geographical coordinates and ratings of all the restaurants in NYC. Then we will display the results on a map to see if some areas have significant higher score.

In [54]:
# The code was removed by Watson Studio for sharing.

In [55]:
import pandas as pd
import requests

# Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&v={}&near={}&limit={}'.format(
    "4d4b7105d754a06374d81259", # We only get venues from the food category
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    "New+York", # We get the venues from NYC
    LIMIT)

# Make the GET request
results = requests.get(url).json()['response']['venues']
#results = requests.get(url).json()['response']['groups'][0]['items']

# Only return relevant information for each nearby venue
venues_list=[]
venues_list.append([(
    v['id'],
    v['name'], 
    v['location']['lat'], 
    v['location']['lng'],  
    v['categories'][0]['name']) for v in results])

NYC_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
NYC_venues.columns = ['id',
                      'name',
                      'latitude',
                      'longitude',
                      'category']

In [56]:
# Get the list of all restaurants in NYC with id, name, coordinates and category
NYC_venues.head()

,id,name,latitude,longitude,category
0,4c5ef77bfff99c74eda954d3,Eataly,40.741987,-73.989913,Gourmet Shop
1,5aebab20336273002c158f04,Serra On The Roof,40.742055,-73.989720,Italian Restaurant
2,40e74880f964a520150a1fe3,Shake Shack,40.741487,-73.988181,Burger Joint
3,59c295349b04732c1d51aa52,Kimbap Lab,40.716160,-73.959530,Korean Restaurant
4,5c71d51689b06a002c6de87d,Au Cheval,40.718075,-74.001974,American Restaurant


In [65]:
# Iterate over rows with iterrows() to add the associated global score
ratings = []
for index, row in NYC_venues.iterrows():
    venue_id = row['id']

    # Prepare request
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id,
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION)
    
    # Make the GET request
    results = requests.get(url).json()
    # If there is no rating for this venue we set the value at 0
    if 'rating' in results['response']['venue']:
        rating = results['response']['venue']['rating']
    else:
        rating = 0
    
    # Add the rating in the ratings dataframe
    ratings.append(rating)

# Add the ratings datagrame as a new column int the venues dataframe
NYC_venues['ratings'] = ratings
NYC_venues.head()

,id,name,latitude,longitude,category,ratings
0,4c5ef77bfff99c74eda954d3,Eataly,40.741987,-73.989913,Gourmet Shop,9.3
1,5aebab20336273002c158f04,Serra On The Roof,40.742055,-73.989720,Italian Restaurant,7.2
2,40e74880f964a520150a1fe3,Shake Shack,40.741487,-73.988181,Burger Joint,9.2
3,59c295349b04732c1d51aa52,Kimbap Lab,40.716160,-73.959530,Korean Restaurant,0.0
4,5c71d51689b06a002c6de87d,Au Cheval,40.718075,-74.001974,American Restaurant,8.6
